# * VINSIGHT : Revenue

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Revenue

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
         
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-29, 10:35:39
   -> v_start_date: 20250101


In [3]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

### Daily Transaction

In [4]:
''' Revenue(Corp) Daily '''

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

rev_corp_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
rev_corp_daily_df = rev_corp_daily_df.loc[rev_corp_daily_df['METRIC_CD'].isin(v_metric_list)]
rev_corp_daily_df = rev_corp_daily_df.loc[rev_corp_daily_df['TM_KEY_MTH']==v_tm_key_mth]

rev_corp_daily_df['TOTAL(Corp)'] = np.where(rev_corp_daily_df['METRIC_CD']=='B0R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['PRE'] = np.where(rev_corp_daily_df['METRIC_CD']=='B1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB1R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['POST'] = np.where(rev_corp_daily_df['METRIC_CD']=='B2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB2R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TOL'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB3R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TVS'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB4R000100', rev_corp_daily_df['C'], 0)

rev_corp_daily_df = rev_corp_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_daily_df = rev_corp_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
rev_corp_daily_df = rev_corp_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

mod_col_list = rev_corp_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_daily_df[col] = rev_corp_daily_df[col].apply(lambda x: format(x, ',.0f'))

rev_corp_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL(Corp),PRE,POST,TOL,TVS
0,202505,20250501,2025-05-29 03:00:52,"185,057,378","175,099,813","9,957,565",0,0
1,202505,20250502,2025-05-29 03:00:52,"1,260,292,612","162,930,833","841,275,888","221,371,077","34,714,813"
2,202505,20250503,2025-05-29 03:00:52,"237,078,095","146,752,230","90,325,865",0,0
3,202505,20250504,2025-05-29 03:00:52,"227,724,497","139,324,000","88,400,498",0,0
4,202505,20250505,2025-05-29 03:00:52,"230,532,922","144,754,994","85,777,928",0,0
5,202505,20250506,2025-05-29 03:00:52,"687,377,331","144,489,760","384,000,243","147,497,449","11,389,879"
6,202505,20250507,2025-05-29 03:00:52,"226,743,747","139,925,621","86,818,126",0,0
7,202505,20250508,2025-05-29 03:00:52,"227,799,230","141,534,062","86,265,167",0,0
8,202505,20250509,2025-05-29 03:00:52,"225,769,562","136,863,210","88,906,352",0,0
9,202505,20250510,2025-05-29 03:00:52,"980,466,375","137,704,908","578,302,706","233,889,879","30,568,883"


In [5]:
''' Revenue(Geo) Daily '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['TM_KEY_MTH']==v_tm_key_mth]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

mod_col_list = rev_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df[col] = rev_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))

rev_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202505,20250501,2025-05-29 03:00:52,"181,113,461","174,932,414","99,880,018","75,052,396","6,181,046",0,"6,181,046",0,0
1,202505,20250502,2025-05-29 03:00:52,"992,937,702","162,783,649","94,043,903","68,739,746","575,568,956","479,993,614","95,575,342","220,543,284","34,041,813"
2,202505,20250503,2025-05-29 03:00:52,"233,704,819","146,582,612","84,480,590","62,102,022","87,122,207",0,"87,122,207",0,0
3,202505,20250504,2025-05-29 03:00:52,"226,737,538","139,197,697","79,916,644","59,281,053","87,539,841",0,"87,539,841",0,0
4,202505,20250505,2025-05-29 03:00:52,"229,099,212","144,651,500","83,827,766","60,823,734","84,447,712",0,"84,447,712",0,0
5,202505,20250506,2025-05-29 03:00:52,"675,602,815","144,364,096","83,847,884","60,516,213","375,080,135","287,826,439","87,253,696","146,710,187","9,448,398"
6,202505,20250507,2025-05-29 03:00:52,"224,161,542","139,653,454","81,677,270","57,976,184","84,508,087",0,"84,508,087",0,0
7,202505,20250508,2025-05-29 03:00:52,"225,753,269","141,525,625","83,755,536","57,770,090","84,227,643",0,"84,227,643",0,0
8,202505,20250509,2025-05-29 03:00:52,"221,456,281","136,844,646","81,516,755","55,327,891","84,611,636",0,"84,611,636",0,0
9,202505,20250510,2025-05-29 03:00:52,"931,271,133","137,684,231","82,117,627","55,566,604","531,228,816","444,332,498","86,896,318","233,389,941","28,968,145"


### Prep Monthly Data

In [6]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display

In [7]:
# ''' Sample Data '''

# # sample_mth_df = monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]
# # sample_mth_df

# sample_day_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()
# sample_day_df.tail(3)

### Special Metric (Corp & Geo)

#### Revenue Corporate (C)
    B0R000100	Total Revenue
    DB0R000100	Total Revenue : DTAC
    TB0R000100	Total Revenue : TRUE
    
    B0R000101	Mobile Revenue
    DB0R000101	Mobile Revenue : DTAC
    TB0R000101	Mobile Revenue : TMH

In [8]:
''' B0R000100 : Total Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'TB1R000100', 'DB2R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00147')

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_df = total_rev_corp_df.loc[total_rev_corp_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_corp_df = total_rev_corp_df.loc[total_rev_corp_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_corp_df['TOTAL'] = np.where(total_rev_corp_df['METRIC_CD']=='B0R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TOL'] = np.where(total_rev_corp_df['METRIC_CD']=='TB3R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TVS'] = np.where(total_rev_corp_df['METRIC_CD']=='TB4R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['CHK_SUM'] = np.where(total_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100','TB3R000100','TB4R000100']), total_rev_corp_df['C'], 0)

total_rev_corp_df = total_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_df['CHK_DIFF'] = total_rev_corp_df['TOTAL'] - total_rev_corp_df['CHK_SUM']
total_rev_corp_df = total_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_df = total_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_df[col] = total_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_T,POST_D,TOL,TVS
0,202501,2025-05-29 03:00:52,"2,759,902,588","12,770,106,773","15,530,009,361","2,444,926,240","1,839,813,343","3,731,079,534","2,739,914,585","1,597,056,733","417,316,338"
1,202502,2025-05-29 03:00:52,"2,400,717,361","12,236,601,216","14,637,318,577","2,217,259,007","1,648,646,062","3,726,430,939","2,727,363,466","1,609,673,918","307,227,824"
2,202503,2025-05-29 03:00:52,"2,734,805,804","12,516,306,281","15,251,112,086","2,466,370,982","1,775,352,571","3,708,553,920","2,734,968,120","1,616,442,724","214,617,965"
3,202504,2025-05-29 03:00:52,0,"12,526,105,797","12,526,105,797","2,366,358,116","1,713,487,323","3,767,633,755","2,744,630,192","1,616,821,911","317,174,500"
4,202505,2025-05-29 03:00:52,0,"10,916,542,924","10,916,542,924","2,117,684,087","1,551,059,202","3,287,985,558","2,331,708,770","1,440,612,645","187,492,662"


In [9]:
''' TB0R000100 : Total Revenue : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100')

v_metric_list = [
    'TB0R000100' #Total Revenue : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_corp_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_true_df = total_rev_corp_true_df.loc[total_rev_corp_true_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_corp_true_df = total_rev_corp_true_df.loc[total_rev_corp_true_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_corp_true_df['TOTAL_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB0R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['PRE_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB1R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['POST_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB2R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TOL'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB3R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TVS'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB4R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['CHK_SUM'] = np.where(total_rev_corp_true_df['METRIC_CD'].isin(['TB1R000100','TB2R000100','TB3R000100','TB4R000100']), total_rev_corp_true_df['C'], 0)

total_rev_corp_true_df = total_rev_corp_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_true_df['CHK_DIFF'] = total_rev_corp_true_df['TOTAL_T'] - total_rev_corp_true_df['CHK_SUM']
total_rev_corp_true_df = total_rev_corp_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_true_df = total_rev_corp_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_T', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_true_df[col] = total_rev_corp_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_T,TOL,TVS
0,202501,2025-05-29 03:00:52,-0,"8,190,378,845","8,190,378,845","2,444,926,240","3,731,079,534","1,597,056,733","417,316,338"
1,202502,2025-05-29 03:00:52,0,"7,860,591,688","7,860,591,688","2,217,259,007","3,726,430,939","1,609,673,918","307,227,824"
2,202503,2025-05-29 03:00:52,0,"8,005,985,591","8,005,985,591","2,466,370,982","3,708,553,920","1,616,442,724","214,617,965"
3,202504,2025-05-29 03:00:52,0,"8,067,988,282","8,067,988,282","2,366,358,116","3,767,633,755","1,616,821,911","317,174,500"
4,202505,2025-05-29 03:00:52,0,"7,033,774,952","7,033,774,952","2,117,684,087","3,287,985,558","1,440,612,645","187,492,662"


In [10]:
''' DB0R000100 : Total Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'DB2R000100')

v_metric_list = [
    'DB0R000100' #Total Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_dtac_df = total_rev_corp_dtac_df.loc[total_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_corp_dtac_df = total_rev_corp_dtac_df.loc[total_rev_corp_dtac_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_corp_dtac_df['TOTAL_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB0R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['PRE_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['POST_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['CHK_SUM'] = np.where(total_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), total_rev_corp_dtac_df['C'], 0)

total_rev_corp_dtac_df = total_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
total_rev_corp_dtac_df['CHK_DIFF'] = total_rev_corp_dtac_df['TOTAL_D'] - total_rev_corp_dtac_df['CHK_SUM']
total_rev_corp_dtac_df = total_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_dtac_df = total_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_D']]

mod_col_list = total_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_dtac_df[col] = total_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_D
0,202501,2025-05-29 03:00:52,"2,759,902,588","4,579,727,928","7,339,630,516","1,839,813,343","2,739,914,585"
1,202502,2025-05-29 03:00:52,"2,400,717,361","4,376,009,529","6,776,726,890","1,648,646,062","2,727,363,466"
2,202503,2025-05-29 03:00:52,"2,734,805,804","4,510,320,691","7,245,126,495","1,775,352,571","2,734,968,120"
3,202504,2025-05-29 03:00:52,0,"4,458,117,515","4,458,117,515","1,713,487,323","2,744,630,192"
4,202505,2025-05-29 03:00:52,0,"3,882,767,972","3,882,767,972","1,551,059,202","2,331,708,770"


In [11]:
''' B0R000101 : Mobile Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'TB2R000100', 'DB1R000100', 'TB1R000100')

v_metric_list = [
    'B0R000101' #Mobile Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_df = mobile_rev_corp_df.loc[mobile_rev_corp_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_corp_df = mobile_rev_corp_df.loc[mobile_rev_corp_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_corp_df['MOBILE'] = np.where(mobile_rev_corp_df['METRIC_CD']=='B0R000101', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['CHK_SUM'] = np.where(mobile_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100']), mobile_rev_corp_df['C'], 0)

mobile_rev_corp_df = mobile_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_df['CHK_DIFF'] = mobile_rev_corp_df['MOBILE'] - mobile_rev_corp_df['CHK_SUM']
mobile_rev_corp_df = mobile_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_df = mobile_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D']]

mod_col_list = mobile_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_df[col] = mobile_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_T,POST_D
0,202501,2025-05-29 03:00:52,"2,759,902,588","10,755,733,702","13,515,636,290","2,444,926,240","1,839,813,343","3,731,079,534","2,739,914,585"
1,202502,2025-05-29 03:00:52,"2,400,717,361","10,319,699,474","12,720,416,836","2,217,259,007","1,648,646,062","3,726,430,939","2,727,363,466"
2,202503,2025-05-29 03:00:52,"2,734,805,804","10,685,245,593","13,420,051,397","2,466,370,982","1,775,352,571","3,708,553,920","2,734,968,120"
3,202504,2025-05-29 03:00:52,0,"10,592,109,386","10,592,109,386","2,366,358,116","1,713,487,323","3,767,633,755","2,744,630,192"
4,202505,2025-05-29 03:00:52,0,"9,288,437,618","9,288,437,618","2,117,684,087","1,551,059,202","3,287,985,558","2,331,708,770"


In [12]:
''' TB0R000101 : Mobile Revenue : TMH '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB2R000100', 'TB1R000100')

v_metric_list = [
    'TB0R000101' #Mobile Revenue : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_corp_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.loc[mobile_rev_corp_tmh_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.loc[mobile_rev_corp_tmh_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_corp_tmh_df['MOBILE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB0R000101', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['PRE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['POST_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['CHK_SUM'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R000100']), mobile_rev_corp_tmh_df['C'], 0)

mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_tmh_df['CHK_DIFF'] = mobile_rev_corp_tmh_df['MOBILE_T'] - mobile_rev_corp_tmh_df['CHK_SUM']
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_T']]

mod_col_list = mobile_rev_corp_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_tmh_df[col] = mobile_rev_corp_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_T
0,202501,2025-05-29 03:00:52,0,"6,176,005,774","6,176,005,774","2,444,926,240","3,731,079,534"
1,202502,2025-05-29 03:00:52,0,"5,943,689,946","5,943,689,946","2,217,259,007","3,726,430,939"
2,202503,2025-05-29 03:00:52,0,"6,174,924,902","6,174,924,902","2,466,370,982","3,708,553,920"
3,202504,2025-05-29 03:00:52,0,"6,133,991,871","6,133,991,871","2,366,358,116","3,767,633,755"
4,202505,2025-05-29 03:00:52,0,"5,405,669,645","5,405,669,645","2,117,684,087","3,287,985,558"


In [13]:
''' DB0R000101 : Mobile Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'DB1R000100')

v_metric_list = [
    'DB0R000101' #Mobile Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.loc[mobile_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.loc[mobile_rev_corp_dtac_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_corp_dtac_df['MOBILE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB0R000101', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['PRE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['POST_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['CHK_SUM'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), mobile_rev_corp_dtac_df['C'], 0)

mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_dtac_df['CHK_DIFF'] = mobile_rev_corp_dtac_df['MOBILE_D'] - mobile_rev_corp_dtac_df['CHK_SUM']
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_D']]

mod_col_list = mobile_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_dtac_df[col] = mobile_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_D
0,202501,2025-05-29 03:00:52,"2,759,902,588","4,579,727,928","7,339,630,516","1,839,813,343","2,739,914,585"
1,202502,2025-05-29 03:00:52,"2,400,717,361","4,376,009,529","6,776,726,890","1,648,646,062","2,727,363,466"
2,202503,2025-05-29 03:00:52,"2,734,805,804","4,510,320,691","7,245,126,495","1,775,352,571","2,734,968,120"
3,202504,2025-05-29 03:00:52,-0,"4,458,117,515","4,458,117,515","1,713,487,323","2,744,630,192"
4,202505,2025-05-29 03:00:52,0,"3,882,767,972","3,882,767,972","1,551,059,202","2,331,708,770"


#### Revenue Geo (P)
    B0R000100GEO	Total Revenue (Geo)
    DB0R000100GEO	Total Revenue (Geo) : DTAC
    TB0R000100GEO	Total Revenue (Geo) : TRUE
    
    B0R000101GEO	Mobile Revenue (Geo)
    DB0R000101GEO	Mobile Revenue (Geo) : DTAC
    TB0R000101GEO	Mobile Revenue (Geo) : TMH

In [14]:
''' B0R000100GEO : Total Revenue (Geo) '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_df = total_rev_geo_df.loc[total_rev_geo_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_geo_df = total_rev_geo_df.loc[total_rev_geo_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_geo_df['TOTAL'] = np.where(total_rev_geo_df['METRIC_CD']=='B0R000100GEO', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TOL'] = np.where(total_rev_geo_df['METRIC_CD']=='TB3R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TVS'] = np.where(total_rev_geo_df['METRIC_CD']=='TB4R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['CHK_SUM'] = np.where(total_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100','TB3R000100','TB4R000100']), total_rev_geo_df['P'], 0)

total_rev_geo_df = total_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_df['CHK_DIFF'] = total_rev_geo_df['TOTAL'] - total_rev_geo_df['CHK_SUM']
total_rev_geo_df = total_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_df = total_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_df[col] = total_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202501,2025-05-29 03:00:52,0,"12,034,347,430","12,034,347,430","2,440,530,082","1,840,026,982","3,336,770,490","2,531,625,556","1,592,511,720","292,882,600"
1,202502,2025-05-29 03:00:52,0,"11,619,121,062","11,619,121,062","2,211,389,140","1,648,818,151","3,333,288,223","2,527,190,353","1,605,419,142","293,016,052"
2,202503,2025-05-29 03:00:52,0,"11,995,980,677","11,995,980,677","2,459,164,272","1,775,434,050","3,319,489,222","2,532,397,179","1,612,256,745","297,239,208"
3,202504,2025-05-29 03:00:52,0,"11,891,585,207","11,891,585,207","2,359,595,474","1,713,499,280","3,371,023,854","2,530,919,910","1,612,742,089","303,804,601"
4,202505,2025-05-29 03:00:52,0,"10,398,151,045","10,398,151,045","2,104,573,386","1,551,064,761","2,987,712,541","2,142,586,581","1,435,175,670","177,038,106"


In [15]:
''' TB0R000100GEO : Total Revenue (Geo) : TRUE '''

v_metric_list = [
    'TB0R000100GEO' #Total Revenue (Geo) : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_geo_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_true_df = total_rev_geo_true_df.loc[total_rev_geo_true_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_geo_true_df = total_rev_geo_true_df.loc[total_rev_geo_true_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_geo_true_df['TOTAL_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB0R000100GEO', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['PRE_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB1R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['POST_B2C_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB2R010100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TOL'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB3R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TVS'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB4R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['CHK_SUM'] = np.where(total_rev_geo_true_df['METRIC_CD'].isin(['TB1R000100','TB2R010100','TB3R000100','TB4R000100']), total_rev_geo_true_df['P'], 0)

total_rev_geo_true_df = total_rev_geo_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_true_df['CHK_DIFF'] = total_rev_geo_true_df['TOTAL_T'] - total_rev_geo_true_df['CHK_SUM']
total_rev_geo_true_df = total_rev_geo_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_true_df = total_rev_geo_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_true_df[col] = total_rev_geo_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,TOL,TVS
0,202501,2025-05-29 03:00:52,0,"7,662,694,892","7,662,694,892","2,440,530,082","3,336,770,490","1,592,511,720","292,882,600"
1,202502,2025-05-29 03:00:52,0,"7,443,112,557","7,443,112,557","2,211,389,140","3,333,288,223","1,605,419,142","293,016,052"
2,202503,2025-05-29 03:00:52,0,"7,688,149,447","7,688,149,447","2,459,164,272","3,319,489,222","1,612,256,745","297,239,208"
3,202504,2025-05-29 03:00:52,0,"7,647,166,017","7,647,166,017","2,359,595,474","3,371,023,854","1,612,742,089","303,804,601"
4,202505,2025-05-29 03:00:52,0,"6,704,499,703","6,704,499,703","2,104,573,386","2,987,712,541","1,435,175,670","177,038,106"


In [16]:
''' DB0R000100GEO : Total Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000100GEO' #Total Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

total_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_dtac_df = total_rev_geo_dtac_df.loc[total_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]
# total_rev_geo_dtac_df = total_rev_geo_dtac_df.loc[total_rev_geo_dtac_df['TM_KEY_MTH']==v_tm_key_mth]

total_rev_geo_dtac_df['TOTAL_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB0R000100GEO', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['PRE_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['POST_B2C_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['CHK_SUM'] = np.where(total_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), total_rev_geo_dtac_df['P'], 0)

total_rev_geo_dtac_df = total_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
total_rev_geo_dtac_df['CHK_DIFF'] = total_rev_geo_dtac_df['TOTAL_D'] - total_rev_geo_dtac_df['CHK_SUM']
total_rev_geo_dtac_df = total_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_dtac_df = total_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = total_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_dtac_df[col] = total_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D
0,202501,2025-05-29 03:00:52,0,"4,371,652,537","4,371,652,537","1,840,026,982","2,531,625,556"
1,202502,2025-05-29 03:00:52,0,"4,176,008,505","4,176,008,505","1,648,818,151","2,527,190,353"
2,202503,2025-05-29 03:00:52,0,"4,307,831,229","4,307,831,229","1,775,434,050","2,532,397,179"
3,202504,2025-05-29 03:00:52,0,"4,244,419,190","4,244,419,190","1,713,499,280","2,530,919,910"
4,202505,2025-05-29 03:00:52,0,"3,693,651,342","3,693,651,342","1,551,064,761","2,142,586,581"


In [17]:
''' B0R000101GEO : Mobile Revenue (Geo) '''

v_metric_list = [
    'B0R000101GEO' #Mobile Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_df = mobile_rev_geo_df.loc[mobile_rev_geo_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_geo_df = mobile_rev_geo_df.loc[mobile_rev_geo_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_geo_df['MOBILE'] = np.where(mobile_rev_geo_df['METRIC_CD']=='B0R000101GEO', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['CHK_SUM'] = np.where(mobile_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100']), mobile_rev_geo_df['P'], 0)

mobile_rev_geo_df = mobile_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_df['CHK_DIFF'] = mobile_rev_geo_df['MOBILE'] - mobile_rev_geo_df['CHK_SUM']
mobile_rev_geo_df = mobile_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_df = mobile_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_df[col] = mobile_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D
0,202501,2025-05-29 03:00:52,0,"10,148,953,110","10,148,953,110","2,440,530,082","1,840,026,982","3,336,770,490","2,531,625,556"
1,202502,2025-05-29 03:00:52,0,"9,720,685,868","9,720,685,868","2,211,389,140","1,648,818,151","3,333,288,223","2,527,190,353"
2,202503,2025-05-29 03:00:52,0,"10,086,484,724","10,086,484,724","2,459,164,272","1,775,434,050","3,319,489,222","2,532,397,179"
3,202504,2025-05-29 03:00:52,0,"9,975,038,517","9,975,038,517","2,359,595,474","1,713,499,280","3,371,023,854","2,530,919,910"
4,202505,2025-05-29 03:00:52,0,"8,785,937,269","8,785,937,269","2,104,573,386","1,551,064,761","2,987,712,541","2,142,586,581"


In [18]:
''' TB0R000101GEO : Mobile Revenue (Geo) : TMH '''

v_metric_list = [
    'TB0R000101GEO' #Mobile Revenue (Geo) : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_geo_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.loc[mobile_rev_geo_tmh_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.loc[mobile_rev_geo_tmh_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_geo_tmh_df['MOBILE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB0R000101GEO', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['PRE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['POST_B2C_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['CHK_SUM'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R010100']), mobile_rev_geo_tmh_df['P'], 0)

mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_tmh_df['CHK_DIFF'] = mobile_rev_geo_tmh_df['MOBILE_T'] - mobile_rev_geo_tmh_df['CHK_SUM']
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_B2C_T']]

mod_col_list = mobile_rev_geo_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_tmh_df[col] = mobile_rev_geo_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_B2C_T
0,202501,2025-05-29 03:00:52,0,"5,777,300,573","5,777,300,573","2,440,530,082","3,336,770,490"
1,202502,2025-05-29 03:00:52,0,"5,544,677,363","5,544,677,363","2,211,389,140","3,333,288,223"
2,202503,2025-05-29 03:00:52,0,"5,778,653,494","5,778,653,494","2,459,164,272","3,319,489,222"
3,202504,2025-05-29 03:00:52,0,"5,730,619,327","5,730,619,327","2,359,595,474","3,371,023,854"
4,202505,2025-05-29 03:00:52,0,"5,092,285,927","5,092,285,927","2,104,573,386","2,987,712,541"


In [19]:
''' DB0R000101GEO : Mobile Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000101GEO' #Mobile Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]
# v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.loc[mobile_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]
# mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.loc[mobile_rev_geo_dtac_df['TM_KEY_MTH']==v_tm_key_mth]

mobile_rev_geo_dtac_df['MOBILE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB0R000101GEO', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['PRE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['POST_B2C_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['CHK_SUM'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), mobile_rev_geo_dtac_df['P'], 0)

mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_dtac_df['CHK_DIFF'] = mobile_rev_geo_dtac_df['MOBILE_D'] - mobile_rev_geo_dtac_df['CHK_SUM']
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_dtac_df[col] = mobile_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_B2C_D
0,202501,2025-05-29 03:00:52,0,"4,371,652,537","4,371,652,537","1,840,026,982","2,531,625,556"
1,202502,2025-05-29 03:00:52,0,"4,176,008,505","4,176,008,505","1,648,818,151","2,527,190,353"
2,202503,2025-05-29 03:00:52,0,"4,307,831,229","4,307,831,229","1,775,434,050","2,532,397,179"
3,202504,2025-05-29 03:00:52,0,"4,244,419,190","4,244,419,190","1,713,499,280","2,530,919,910"
4,202505,2025-05-29 03:00:52,0,"3,693,651,342","3,693,651,342","1,551,064,761","2,142,586,581"


### Revenue by Products

#### All Service

In [20]:
''' All Service '''

v_product_grp = 'All Services'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

all_service_df = monthly_df_display.copy()
all_service_df = all_service_df.loc[all_service_df['PRODUCT_GRP']==v_product_grp]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,All Services,B0R000100,Total Revenue,2025-05-28 23:53:59,"10,916,542,924","10,887,546,250",0,0,0,"28,996,674"
1,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,0,"10,398,151,045","10,216,783,978","10,216,524,898","10,216,538,082","-10,398,151,045"
2,202505,All Services,DB0R000100,Total Revenue : DTAC,2025-05-28 23:53:59,"3,882,767,972","3,882,773,531",0,0,0,"-5,559"
3,202505,All Services,DB0R000100GEO,Total Revenue (Geo) : DTAC,2025-05-29 03:00:52,0,"3,693,651,342","3,693,650,490","3,693,565,442","3,693,565,442","-3,693,651,342"
4,202505,All Services,TB0R000100,Total Revenue : TRUE,2025-05-28 23:53:59,"7,033,774,952","7,004,772,719",0,0,0,"29,002,233"
5,202505,All Services,TB0R000100GEO,Total Revenue (Geo) : TRUE,2025-05-29 03:00:52,0,"6,704,499,703","6,523,133,488","6,522,959,456","6,522,972,641","-6,704,499,703"


#### Mobile

In [21]:
''' Mobile '''

v_product_grp = 'Mobile'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

mobile_df = monthly_df_display.copy()
mobile_df = mobile_df.loc[mobile_df['PRODUCT_GRP']==v_product_grp]
mobile_df = mobile_df.loc[mobile_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_df = mobile_df.reset_index(drop=True)
mobile_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Mobile,B0R000101,Mobile Revenue,2025-05-28 23:53:59,"9,288,437,618","9,275,332,474",0,0,0,"13,105,143"
1,202505,Mobile,B0R000101GEO,Mobile Revenue (Geo),2025-05-29 03:00:52,0,"8,785,937,269","8,604,570,202","8,603,958,270","8,603,958,270","-8,785,937,269"
2,202505,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-28 23:53:59,"3,882,767,972","3,882,773,531",0,0,0,"-5,559"
3,202505,Mobile,DB0R000101GEO,Mobile Revenue (Geo) : DTAC,2025-05-29 03:00:52,0,"3,693,651,342","3,693,650,490","3,693,565,442","3,693,565,442","-3,693,651,342"
4,202505,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-28 23:53:59,"5,405,669,645","5,392,558,943",0,0,0,"13,110,702"
5,202505,Mobile,TB0R000101GEO,Mobile Revenue (Geo) : TMH,2025-05-29 03:00:52,0,"5,092,285,927","4,910,919,712","4,910,392,829","4,910,392,829","-5,092,285,927"


#### Prepaid

In [22]:
''' Prepaid '''

v_product_grp = 'Prepaid'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

prepaid_df = monthly_df_display.copy()
prepaid_df = prepaid_df.loc[prepaid_df['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Prepaid,B1R000100,Prepaid Revenue,2025-05-29 03:00:52,"3,668,743,290","3,655,638,147","3,474,280,575","3,473,679,418","3,473,679,418","13,105,143"
1,202505,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-05-29 03:00:52,"1,551,059,202","1,551,064,761","1,551,064,761","1,550,988,015","1,550,988,015","-5,559"
2,202505,Prepaid,TB1R000100,Prepaid Revenue : TMH,2025-05-29 03:00:52,"2,117,684,087","2,104,573,386","1,923,215,814","1,922,691,403","1,922,691,403","13,110,702"


#### Postpaid

In [23]:
''' Postpaid '''

v_product_grp = 'Postpaid'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

postpaid_df = monthly_df_display.copy()
postpaid_df = postpaid_df.loc[postpaid_df['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2R000100,Postpaid Revenue,2025-05-28 23:53:59,"5,437,900,976","5,437,900,976",0,0,0,0
1,202505,Postpaid,B2R010100,Postpaid Revenue B2C,2025-05-29 03:00:52,"4,955,205,180","4,955,205,180","4,955,195,884","4,955,185,469","4,955,185,469",0
2,202505,Postpaid,B2R020100,Postpaid Revenue B2B,2025-05-28 23:53:59,"482,695,449","482,695,449",0,0,0,0
3,202505,Postpaid,DB2R000100,Postpaid Revenue : DTAC,2025-05-29 03:00:52,"2,331,708,770","2,331,708,770","2,331,707,919","2,330,661,634","2,330,661,634",0
4,202505,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-05-29 03:00:52,"2,142,586,581","2,142,586,581","2,142,585,729","2,142,577,427","2,142,577,427",0
5,202505,Postpaid,DB2R020100,Postpaid Revenue B2B : DTAC,2025-05-29 03:00:52,"189,122,190","189,122,190","189,122,190","188,084,207","188,084,207",0
6,202505,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-05-28 23:53:59,"3,287,985,558","3,287,985,558",0,0,0,0
7,202505,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,"2,987,712,541","2,987,712,541","2,987,703,898","2,987,701,426","2,987,701,426",0
8,202505,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-05-28 23:53:59,"300,272,670","300,272,670",0,0,0,0


#### TOL

In [24]:
''' TOL '''

v_product_grp = 'TOL'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

tol_df = monthly_df_display.copy()
tol_df = tol_df.loc[tol_df['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,TOL,TB3R000100,TOL Revenue,2025-05-29 03:00:52,"1,440,612,645","1,435,175,670","1,435,175,670","1,435,175,670","1,435,175,670","5,436,975"


#### TVS

In [25]:
''' TVS '''

v_product_grp = 'TVS'
v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

tvs_df = monthly_df_display.copy()
tvs_df = tvs_df.loc[tvs_df['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202505,TVS,TB4R000100,TVS Revenue,2025-05-29 03:00:52,"187,492,662","177,038,106","177,038,106","177,390,958","177,404,142","10,454,556"
